In [ ]:
import pygame
import sys
from Player import Player  # Import Player from external file
from CircuitSimulator import CircuitSimulator
from GameUI import GameUI

from qiskit import QuantumCircuit
from qiskit_aer.noise import NoiseModel, depolarizing_error

# --- Game Setup ---

pygame.init()
WIDTH, HEIGHT = 1500, 600  
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Quantum Gate Minigame")
font = pygame.font.SysFont(None, 32)
players = [Player("Mario"), Player("Luigi"), Player("Yoshi"), Player("Peach")]
current_player = 0
skipped_players = set()

# --- Gate UI Setup ---

GATE_COLORS = {"H": (200,200,255),"Z": (255,200,200),"Y": (200,255,200),"X": (255,255,200),"CNOT": (200,255,255),"SWAP": (255,200,255), "DECOH": (120,120,120)}
GATE_LIST = ["H","Z","Y","X","CNOT","SWAP"]
gate_rects = {}
for i, gate in enumerate(GATE_LIST):
    gate_rects[gate] = pygame.Rect(30, 50 + i*60, 80, 40)

# --- Circuit State ---
MAX_GATES = 20

qiskit_circuit = CircuitSimulator.create_empty_circuit()
gate_history = [("H", 0), ("H", 1, "layer0")]  

# --- Decoherence State ---

def get_decoherence_percent():
    # Decoherence is now 10% per DECOH gate (max 100)
    decoh_gates = [g for g in gate_history if g[0] == "DECOH"]
    percent = min(100, len(decoh_gates) * 20)
    return percent


# --- Drag and Drop ---
dragging_gate = None
drag_offset = (0,0)
drag_pos = (0,0)

# --- Main Loop ---

measurement_result = None

def next_player():
    global current_player
    n = len(players)
    for _ in range(n):
        current_player = (current_player + 1) % n
        if current_player not in skipped_players:
            return

while True:
    screen.fill((240,240,240))
    GameUI.draw_player_info(screen, font, players, current_player)
    GameUI.draw_gates(screen, font, players[current_player], players, current_player, gate_rects, GATE_LIST, GATE_COLORS)
    GameUI.draw_circuit(screen, font, gate_history, GATE_COLORS, MAX_GATES)
    btn_rect = GameUI.draw_measure_button(screen, font, WIDTH, HEIGHT)
    skip_btn_rect = GameUI.draw_skip_button(screen, font, WIDTH, HEIGHT)
    # Draw decoherence percent
    
    decoh_percent = get_decoherence_percent()

    decoh_txt = font.render(f"Decoherence chance: {decoh_percent}%", True, (120,0,0))

    screen.blit(decoh_txt, (WIDTH-350, HEIGHT-180))
    if measurement_result:
        txt = font.render(f"Measured: {measurement_result}", True, (0,0,0))
        screen.blit(txt, (WIDTH-300, HEIGHT-140))
    # Draw dragging gate if any
    if dragging_gate:
        mx, my = drag_pos
        pygame.draw.rect(screen, GATE_COLORS[dragging_gate], (mx-40, my-20, 80, 40))
        txt = font.render(dragging_gate, True, (0,0,0))
        screen.blit(txt, (mx-20, my-10))
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            if skip_btn_rect.collidepoint(mx, my):
                skipped_players.add(current_player)
                next_player()
                dragging_gate = None
                drag_pos = (0,0)
                continue
            for gate, rect in gate_rects.items():
                if rect.collidepoint(mx, my) and players[current_player].gates[gate] > 0:
                    dragging_gate = gate
                    drag_offset = (mx - rect.x, my - rect.y)
                    drag_pos = (mx, my)
            if btn_rect.collidepoint(mx, my):
                # Apply decoherence noise model
                percent = get_decoherence_percent()
                noise_model = CircuitSimulator.apply_decoherence_noise(qiskit_circuit, percent)
                measurement_result = CircuitSimulator.apply_circuit(qiskit_circuit, noise_model=noise_model,gate_history=gate_history)
        elif event.type == pygame.MOUSEBUTTONUP:
            if dragging_gate:
                mx, my = event.pos
                base_x = 200
                gate_layer = max(0, len(gate_history) - 2)
                drop_x = base_x + (gate_layer+2)*60
                dropped = False
                for q in range(2):
                    y = 150 + q*60
                    if drop_x-20 < mx < drop_x+20 and y-20 < my < y+20:
                        if dragging_gate == "CNOT":
                            control = q
                            target = 1 - q
                            qiskit_circuit.cx(control, target)
                            gate_history.append(("CNOT", control, target))
                            players[current_player].gates["CNOT"] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                        elif dragging_gate == "SWAP":
                            qiskit_circuit.swap(0, 1)
                            gate_history.append(("SWAP", 0, 1))
                            players[current_player].gates["SWAP"] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                        else:
                            if dragging_gate == "H":
                                qiskit_circuit.h(q)
                            elif dragging_gate == "X":
                                qiskit_circuit.x(q)
                            elif dragging_gate == "Y":
                                qiskit_circuit.y(q)
                            elif dragging_gate == "Z":
                                qiskit_circuit.z(q)
                            gate_history.append((dragging_gate, q))
                            players[current_player].gates[dragging_gate] -= 1
                            next_player()
                            measurement_result = None
                            dropped = True
                            break
                # Insert decoherence gate every 4 user-placed gates
                placed_gates = [g for g in gate_history[2:] if g[0] != "DECOH"]
                if len(placed_gates) > 0 and len(placed_gates) % 4 == 0:
                     gate_history.append(("DECOH", None))
                dragging_gate = None
                drag_pos = (0,0)
        elif event.type == pygame.MOUSEMOTION:
            if dragging_gate:
                drag_pos = event.pos

SystemExit: 